In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
#pytorch 
import torch

#python lib
import random
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

#detectron2 lib
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.visualizer import Visualizer



#computer vision 
import cv2


In [ ]:
class CONFIG:
    seed = 42
    trained_weights_path = "../input/sartorius-models/model_final.pth"
    
    # detetron2 model zoo model
    model_zoo_model ="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    
THRESHOLDS = [.15, .35, .55]
MIN_PIXELS = [75, 150, 75]

In [ ]:
def seed_torch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def seed_python(seed):
    random.seed(seed)
    np.random.seed(seed)
    
seed_python(CONFIG.seed)
seed_torch(CONFIG.seed)

In [ ]:
#Helper functions
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def get_masks(fn, predictor):
    im = cv2.imread(str(fn))
    pred = predictor(im)
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= MIN_PIXELS[pred_class]: # skip predictions with small area
            used += mask
            res.append(rle_encode(mask))
    return res

In [ ]:
## Getting the images
test_images_paths = glob.glob("../input/sartorius-cell-instance-segmentation/test/*.png")
test_images_id = [os.path.splitext(os.path.basename(path))[0] for path in test_images_paths]

In [ ]:
## loading model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG.model_zoo_model))
cfg.INPUT.MASK_FORMAT = 'bitmask'
cfg.MODEL.WEIGHTS = "../input/satorius-train-detectron2-models/model_0007999.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
predictor = DefaultPredictor(cfg)

In [ ]:
## Sanity check
# test_image = cv2.imread(test_images_paths[0])
# output = predictor(test_image)
# visualizer = Visualizer(test_image[:,:,::-1], scale=2.0)
# out = visualizer.draw_instance_predictions(output['instances'].to("cpu"))
# plt.figure(figsize=(20,15))
# plt.imshow(out.get_image()[:,:,::-1])

In [ ]:
# encoded_masks = get_masks(test_images_paths[0], predictor)

# _, axs = plt.subplots(1,2, figsize=(40,15))
# axs[1].imshow(cv2.imread(str(test_images_paths[0])))
# for enc in encoded_masks:
#     dec = rle_decode(enc)
#     axs[0].imshow(np.ma.masked_where(dec==0, dec))

In [ ]:
ids = []
masks = []
for fn, idx in zip(test_images_paths , test_images_id):
    encoded_masks = get_masks(fn, predictor)
    for enc in encoded_masks:
        ids.append(idx)
        masks.append(enc)
        
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()